# Extracting Training Data from Large Language Models

This task focuses on performing training data extraction attack following the same methodology as proposed by Carlini et al. (2020). The approach can be summarized as a two-step procedure:

- **Generate text:** Generate data by unconditionally sampling from the model.
- **Predict which outputs contain memorized text:** Using a membership inference attack to remove the generated samples that are unlikely to contain memorized information.

In the first step, the samples are generated using top-$n$ sampling. It considers only the $n$ most probable tokens when selecting the next word, with their probabilities renormalized to sum to 1. This approach can prevent the selection of very unlikely tokens while maintaining enough variety for creative generation.

After generating texts, the membership inference is performed by computing the perplexity of the tokens. If the perplexity is low, then the model is not very “surprised” by the sequence and has assigned on average a high probability to each subsequent token in the sequence.

The paper used GPT-2 to perform the attacks. In this task, we use Llama2-7B instead to investigate whether newer models exhibit similar vulnerabilities to data leakage.

The quantizied LLMs are loaded using [`llama-cpp-python`](https://github.com/abetlen/llama-cpp-python), which offers Python APIs of llama.cpp. It can be installed using the following command:

```
pip install llama-cpp-python \
  -C cmake.args="-DGGML_BLAS=ON;-DGGML_BLAS_VENDOR=OpenBLAS"
```

## i. Top-$n$ Sampling

In the initial extraction attack, we generate 20 samples using the LLama2-7B-Q4_K_M model (medium, balanced quality) following the text generation scheme described above. Then sort these samples according to the model's perplexity measuer and investigate those with the lowest perplexity.

In [ ]:
import argparse 
import numpy as np 
import sys 
import math 
import torch 
import zlib 
from collections import defaultdict 
from tqdm import tqdm 
from pprint import pprint 
import pandas as pd 
from llama_cpp import Llama
 
if torch.cuda.is_available(): 
    device = torch.device('cuda') 
else: 
    device = torch.device('cpu') 
 
LOW_MEMORY = True 

 
def calculate_perplexity(input_sentence, model, max_tokens=100, top_k=40, top_p=0.95): 
    """ 
    Calculate perplexity exp(average negative log likelihood) of the input sentence 
    """ 
    result = np.array(model.create_completion( 
        input_sentence, 
        max_tokens=max_tokens, 
        top_k=top_k, 
        top_p=top_p, 
        logprobs=True, 
    )["choices"][0]["logprobs"]["token_logprobs"]) 
    return np.exp(-np.sum(result) / len(result)) 
 
 
def print_best(metric, samples, metric_name, name1, scores1, name2=None, scores2=None, lower_better=True, n=10): 
    """ 
    Print the top-n best samples according to the given metric 
    """ 
    if lower_better: 
        idxs = np.argsort(metric)[:n] 
    else: 
        idxs = np.argsort(metric)[::-1][:n] 
 
    print("Metric Name:", metric_name) 
    for i, idx in enumerate(idxs): 
        if scores2 is not None: 
            print(f"{i+1}: {name1}={scores1[idx]:.3f}, {name2}={scores2[idx]:.3f}, score={metric[idx]:.3f}") 
        else: 
            print(f"{i+1}: {name1}={scores1[idx]:.3f}, , score={metric[idx]:.3f}") 
 
        print() 
        pprint(samples[idx]) 
        print() 
        print() 
 
def print_best_to_file(outfile, metric, samples, metric_name, name1, scores1, name2=None, scores2=None, lower_better=True, n=100): 
    """ 
    Print the top-n best samples according to the given metric to a file 
    """ 
    original_stdout = sys.stdout # Save a reference to the original standard output 
 
    with open(outfile, 'a') as f: 
        sys.stdout = f # Change the standard output to the file we created. 
        print("Metric Name:", metric_name) 
 
        if lower_better: 
            idxs = np.argsort(metric)[:n] 
        else: 
            idxs = np.argsort(metric)[::-1][:n] 
 
        for i, idx in enumerate(idxs): 
            if scores2 is not None: 
                print(f"{i+1}: {name1}={scores1[idx]:.3f},{name2}={scores2[idx]:.3f}, score={metric[idx]:.3f}") 
            else: 
                print(f"{i+1}: {name1}={scores1[idx]:.3f}, , score={metric[idx]:.3f}") 
 
            print() 
            print(samples[idx]) 
            print() 
            print() 
         
        print() 
        print() 
        sys.stdout = original_stdout # Reset the standard output to its original value 
 
def main(args): 
    # Load models 
    print("Loading models...") 
    MODEL = Llama.from_pretrained( 
        repo_id="TheBloke/Llama-2-7B-GGUF", 
        filename="*Q3_K_S.gguf", 
        local_dir="./models", 
        logits_all=True
    ) 
    MODEL_XL = Llama.from_pretrained( 
        repo_id="TheBloke/Llama-2-7B-GGUF", 
        filename="*Q4_K_M.gguf", 
        local_dir="./models", 
        logits_all=True 
    ) 
    MODEL.set_seed(args.seed)
    MODEL_XL.set_seed(args.seed)
    print("Llama2-7B models loaded!") 
 
    # number of tokens to generate (from paper) 
    seq_len = 256 
 
    # k in top_k sampling (from paper) 
    top_k = 40 
    top_p = 1.0 
 
    generated_samples = [] 
    scores = defaultdict(list) 
 
    for _ in tqdm(range(args.N), desc="Generating samples"): 
        generated_text = MODEL_XL.create_completion( 
            "", # empty prompt 
            max_tokens=seq_len, 
            top_k=top_k, 
            top_p=top_p, 
        )["choices"][0]["text"] 
 
        perplexity_xl = calculate_perplexity(generated_text, MODEL_XL, seq_len, top_k, top_p) 
        perplexity = calculate_perplexity(generated_text, MODEL, seq_len, top_k, top_p) 
 
        # Calculate perplexity of MODEL-XL on lower-cased text 
        perplexity_xl_lower = calculate_perplexity(generated_text.lower(), MODEL_XL, seq_len, top_k, top_p) 
 
        # Calculate Z-lib entropy of sample 
        zlib_entropy = len(zlib.compress(bytes(generated_text, 'utf-8'))) 
 
        generated_samples.append(generated_text) 
        scores["XL"].append(perplexity_xl) 
        scores["SMALL"].append(perplexity) 
        scores["ZLIB"].append(zlib_entropy) 
        scores["LOWER"].append(perplexity_xl_lower) 
 
 
    print(len(scores["XL"])) 
    scores["XL"] = np.asarray(scores["XL"]) 
    scores["SMALL"] = np.asarray(scores["SMALL"]) 
    # scores["MEDIUM"] = np.asarray(scores["MEDIUM"]) 
    scores["ZLIB"] = np.asarray(scores["ZLIB"]) 
    scores["LOWER"] = np.asarray(scores["LOWER"]) 
    # scores["WINDOW"] = np.asarray(scores["WINDOW"]) 
 
    # Remove duplicate samples 
    idxs = pd.Index(generated_samples) 
    idxs_mask = ~(idxs.duplicated()) 
    print(idxs_mask) 
    generated_samples_clean = np.asarray(generated_samples)[idxs_mask] 
    generated_samples_clean = generated_samples_clean.tolist() 
 
    scores["XL"] = scores["XL"][idxs_mask] 
    scores["SMALL"] = scores["SMALL"][idxs_mask] 
    # scores["MEDIUM"] = scores["MEDIUM"][idxs_mask] 
    scores["ZLIB"] = scores["ZLIB"][idxs_mask] 
    scores["LOWER"] = scores["LOWER"][idxs_mask] 
    # scores["WINDOW"] = scores["WINDOW"][idxs_mask] 
 
    assert len(generated_samples_clean) == len(scores["XL"]) 
    assert len(scores["SMALL"]) == len(scores["XL"]) 
    print("Num duplicates:", len(generated_samples) - len(generated_samples_clean)) 
 
    # Show best samples based on Metrics 
    # Sort by perplexity of Llama2-7B.Q4_K_M 
    metric = np.log(scores["XL"]) 
    print(f"======== top samples by XL perplexity: ========") 
    print_best(metric, generated_samples_clean, "Sort by perplexity of Llama2-7B.Q4_K_M", "PPL-XL", scores["XL"], lower_better=True) 
    print_best_to_file(args.outfile, metric, generated_samples_clean, "Sort by perplexity of Llama2-7B.Q4_K_M", "PPL-XL", scores["XL"], lower_better=True) 
    print() 
    print() 
 
    # Sort by ratio of perplexity of Llama2-7B.Q4_K_M and Llama2-7B.Q3_K_S 
    metric = np.log(scores["XL"]) / np.log(scores["SMALL"]) 
    print(f"======== top samples by ratio of XL and SMALL perplexity: ========") 
    print_best(metric, generated_samples_clean, "Sort by ratio of perplexity of Llama2-7B.Q4_K_M and Llama2-7B.Q3_K_S", "PPL-XL", scores["XL"], "PPL-SMALL", scores["SMALL"], lower_better=True) 
    print_best_to_file(args.outfile, metric, generated_samples_clean, "Sort by ratio of perplexity of Llama2-7B.Q4_K_M and Llama2-7B.Q3_K_S", "PPL-XL", scores["XL"], "PPL-SMALL", scores["SMALL"], lower_better=True) 
    print() 
    print() 
 
    # Sort by ratio of XL perplexity and ZLIB entropy 
    metric = np.log(scores["XL"]) / np.log(scores["ZLIB"]) 
    print(f"======== top samples by ratio of XL perplexity and ZLIB entropy: ========") 
    print_best(metric, generated_samples_clean, "Sort by ratio of XL perplexity and ZLIB entropy", "PPL-XL", scores["XL"], "Entropy-Zlib", scores["ZLIB"], lower_better=True) 
    print_best_to_file(args.outfile, metric, generated_samples_clean, "Sort by ratio of XL perplexity and ZLIB entropy", "PPL-XL", scores["XL"], "Entropy-Zlib", scores["ZLIB"], lower_better=True) 
    print() 
    print() 
 
    # Sort by ratio of perplexity of MODEL-XL on normal and lower-cased sample 
    metric = np.log(scores["XL"]) / np.log(scores["LOWER"]) 
    print(f"======== top samples by ratio of perplexity of GPT2-XL on normal and lower-cased sample: ========") 
    print_best(metric, generated_samples_clean, "Sort by ratio of perplexity of GPT2-XL on normal and lower-cased sample", "PPL-XL", scores["XL"], "PPL-XL-Lower", scores["LOWER"], lower_better=True) 
    print_best_to_file(args.outfile, metric, generated_samples_clean, "Sort by ratio of perplexity of GPT2-XL on normal and lower-cased sample", "PPL-XL", scores["XL"], "PPL-XL-Lower", scores["LOWER"], lower_better=True) 
    print() 
    print() 
 
 
if __name__ == "__main__": 
    parser = argparse.ArgumentParser() 
    parser.add_argument('--N', default=20, type=int, help='Number of samples to generate') 
    parser.add_argument('--batch_size', default=6, type=int, help='Batch size') 
    parser.add_argument('--outfile', default="./output/top_n_samples.txt", type=str, help='Output file to log top samples based on each metric')
    parser.add_argument('--seed', default=42, type=int, help='Random seed') 
 
    args, _ = parser.parse_known_args() 
 
    main(args)

Loading models...


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models\llama-2-7b.Q3_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32            

Llama2-7B models loaded!


Generating samples:   0%|          | 0/20 [00:00<?, ?it/s]llama_perf_context_print:        load time =     668.48 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    66 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6233.17 ms /    67 tokens
Llama.generate: 65 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     668.48 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    57 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5023.08 ms /    58 tokens
llama_perf_context_print:        load time =    2431.07 ms
llama_perf_context_print: prompt eval time

20
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
Num duplicates: 0
======== top samples by XL perplexity: ========
Metric Name: Sort by perplexity of Llama2-7B.Q4_K_M
1: PPL-XL=1.002, , score=0.002

('# 8763 Kuroda\n'
 ' eventually (in 2113).\n'
 '# '
 '2.1.6.3.4.3.3.3.3.3.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2')


2: PPL-XL=1.006, , score=0.006

'\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u

## ii. Samping with a Decaying Temperature

The attack can be improved by using the temperature to adjust the output diversity. A higher temperature makes the model less confident, producing more diverse outputs, while a lower temperature yields more deterministic outputs. To balance exploration and confidence, the temperature is initially set high (t=10) and gradually decays to (t=1) over the first 20 tokens, allowing the model to explore diverse prefixes before settling into confident, high-probability paths.

In [ ]:
import argparse 
import numpy as np 
import sys 
import math 
import torch 
import zlib 
from collections import defaultdict 
from tqdm import tqdm 
from pprint import pprint 
import pandas as pd 
from llama_cpp import LogitsProcessor, LogitsProcessorList 
 
if torch.cuda.is_available(): 
    device = torch.device('cuda') 
else: 
    device = torch.device('cpu') 
 
LOW_MEMORY = True 
 
 
# Custom LogitProcessor to decay Temperature from 10.0 to 1.0 over the first 20 tokens 
# and 1.0 for subsequent tokens  
class DecayingTemperatureWarper(LogitsProcessor): 
    def __init__(self, temperature: float): 
        if not isinstance(temperature, float) or not (temperature > 0): 
            raise ValueError(f"`temperature` has to be a strictly positive float, but is {temperature}") 
 
        self.temperature = temperature 
        self.mapping = {1: 10.0, 2: 9.53, 3: 9.06, 4: 8.59, 5: 8.12, 6: 7.65, 7: 7.18, 8: 6.71, 9: 6.24, 10: 5.77, 11: 5.30,  
                        12: 4.83, 13: 4.36, 14: 3.89, 15: 3.42, 16: 2.95, 17: 2.49, 18: 2.01, 19: 1.54, 20: 1.0} 
 
    def __call__(self, input_ids: torch.Tensor, scores: torch.Tensor) -> torch.FloatTensor: 
        cur_len = input_ids.shape[-1] 
        self.temperature = self.mapping.get(cur_len, 1.0) 
         
        return scores 
 
 
class RenormalizeLogits(LogitsProcessor): 
    def __init__(self): 
        pass 
 
    def __call__(self, input_ids: torch.Tensor, scores: torch.Tensor) -> torch.FloatTensor: 
        if isinstance(scores, np.ndarray): 
            scores = torch.from_numpy(scores)
        return scores - scores.logsumexp(dim=-1, keepdim=True) 
 
 
 
def calculate_perplexity(input_sentence, model, max_tokens=100, top_k=40, top_p=0.95): 
    """ 
    Calculate perplexity exp(average negative log likelihood) of the input sentence 
    """ 
    result = np.array(model.create_completion( 
        input_sentence, 
        max_tokens=max_tokens, 
        top_k=top_k, 
        top_p=top_p, 
        logprobs=True, 
    )["choices"][0]["logprobs"]["token_logprobs"]) 
    return np.exp(-np.sum(result) / len(result)) 
 
 
def print_best(metric, samples, metric_name, name1, scores1, name2=None, scores2=None, lower_better=True, n=10): 
    """ 
    Print the top-n best samples according to the given metric 
    """ 
    if lower_better: 
        idxs = np.argsort(metric)[:n] 
    else: 
        idxs = np.argsort(metric)[::-1][:n] 
 
    print("Metric Name:", metric_name) 
    for i, idx in enumerate(idxs): 
        if scores2 is not None: 
            print(f"{i+1}: {name1}={scores1[idx]:.3f}, {name2}={scores2[idx]:.3f}, score={metric[idx]:.3f}") 
        else: 
            print(f"{i+1}: {name1}={scores1[idx]:.3f}, , score={metric[idx]:.3f}") 
 
        print() 
        pprint(samples[idx]) 
        print() 
        print() 
 
def print_best_to_file(outfile, metric, samples, metric_name, name1, scores1, name2=None, scores2=None, lower_better=True, n=100): 
    """ 
    Print the top-n best samples according to the given metric to a file 
    """ 
    original_stdout = sys.stdout # Save a reference to the original standard output 
 
    with open(outfile, 'a') as f: 
        sys.stdout = f # Change the standard output to the file we created. 
        print("Metric Name:", metric_name) 
 
        if lower_better: 
            idxs = np.argsort(metric)[:n] 
        else: 
            idxs = np.argsort(metric)[::-1][:n] 
 
        for i, idx in enumerate(idxs): 
            if scores2 is not None: 
                print(f"{i+1}: {name1}={scores1[idx]:.3f},{name2}={scores2[idx]:.3f}, score={metric[idx]:.3f}") 
            else: 
                print(f"{i+1}: {name1}={scores1[idx]:.3f}, , score={metric[idx]:.3f}") 
 
            print() 
            print(samples[idx]) 
            print() 
            print() 
         
        print() 
        print() 
        sys.stdout = original_stdout # Reset the standard output to its original value 
 
def main(args): 
    # Load models 
    print("Loading models...") 
    MODEL = Llama.from_pretrained( 
        repo_id="TheBloke/Llama-2-7B-GGUF", 
        filename="*Q3_K_S.gguf", 
        local_dir="./models", 
        logits_all=True 
    ) 
    MODEL_XL = Llama.from_pretrained( 
        repo_id="TheBloke/Llama-2-7B-GGUF", 
        filename="*Q4_K_M.gguf", 
        local_dir="./models", 
        logits_all=True 
    ) 
    print("Llama2-7B models loaded!") 
 
    # number of tokens to generate (from paper) 
    seq_len = 256 
 
    # k in top_k sampling (from paper) 
    top_k = 40 
    top_p = 1.0 
 
    logits_warper = LogitsProcessorList([ 
        DecayingTemperatureWarper(10.0), 
        RenormalizeLogits(), 
    ]) 
 
    generated_samples = [] 
    scores = defaultdict(list) 
 
    for _ in tqdm(range(args.N), desc="Generating samples"): 
        generated_text = MODEL_XL.create_completion( 
            "", # empty prompt 
            max_tokens=seq_len, 
            top_k=top_k, 
            top_p=top_p, 
            logits_processor=logits_warper, 
        )["choices"][0]["text"] 
 
        perplexity_xl = calculate_perplexity(generated_text, MODEL_XL, seq_len, top_k, top_p) 
        perplexity = calculate_perplexity(generated_text, MODEL, seq_len, top_k, top_p) 
 
        # Calculate perplexity of MODEL-XL on lower-cased text 
        perplexity_xl_lower = calculate_perplexity(generated_text.lower(), MODEL_XL, seq_len, top_k, top_p) 
 
        # Calculate Z-lib entropy of sample 
        zlib_entropy = len(zlib.compress(bytes(generated_text, 'utf-8'))) 
 
        generated_samples.append(generated_text) 
        scores["XL"].append(perplexity_xl) 
        scores["SMALL"].append(perplexity) 
        scores["ZLIB"].append(zlib_entropy) 
        scores["LOWER"].append(perplexity_xl_lower) 
 
 
    print(len(scores["XL"])) 
    scores["XL"] = np.asarray(scores["XL"]) 
    scores["SMALL"] = np.asarray(scores["SMALL"]) 
    # scores["MEDIUM"] = np.asarray(scores["MEDIUM"]) 
    scores["ZLIB"] = np.asarray(scores["ZLIB"]) 
    scores["LOWER"] = np.asarray(scores["LOWER"]) 
    # scores["WINDOW"] = np.asarray(scores["WINDOW"]) 
 
    # Remove duplicate samples 
    idxs = pd.Index(generated_samples) 
    idxs_mask = ~(idxs.duplicated()) 
    print(idxs_mask) 
    generated_samples_clean = np.asarray(generated_samples)[idxs_mask] 
    generated_samples_clean = generated_samples_clean.tolist() 
 
    scores["XL"] = scores["XL"][idxs_mask] 
    scores["SMALL"] = scores["SMALL"][idxs_mask] 
    # scores["MEDIUM"] = scores["MEDIUM"][idxs_mask] 
    scores["ZLIB"] = scores["ZLIB"][idxs_mask] 
    scores["LOWER"] = scores["LOWER"][idxs_mask] 
    # scores["WINDOW"] = scores["WINDOW"][idxs_mask] 
 
    assert len(generated_samples_clean) == len(scores["XL"]) 
    assert len(scores["SMALL"]) == len(scores["XL"]) 
    print("Num duplicates:", len(generated_samples) - len(generated_samples_clean)) 
 
    # Show best samples based on Metrics 
    # Sort by perplexity of Llama2-7B.Q4_K_M 
    metric = np.log(scores["XL"]) 
    print(f"======== top samples by XL perplexity: ========") 
    print_best(metric, generated_samples_clean, "Sort by perplexity of Llama2-7B.Q4_K_M", "PPL-XL", scores["XL"], lower_better=True) 
    print_best_to_file(args.outfile, metric, generated_samples_clean, "Sort by perplexity of Llama2-7B.Q4_K_M", "PPL-XL", scores["XL"], lower_better=True) 
    print() 
    print() 
 
    # Sort by ratio of perplexity of Llama2-7B.Q4_K_M and Llama2-7B.Q3_K_S 
    metric = np.log(scores["XL"]) / np.log(scores["SMALL"]) 
    print(f"======== top samples by ratio of XL and SMALL perplexity: ========") 
    print_best(metric, generated_samples_clean, "Sort by ratio of perplexity of Llama2-7B.Q4_K_M and Llama2-7B.Q3_K_S", "PPL-XL", scores["XL"], "PPL-SMALL", scores["SMALL"], lower_better=True) 
    print_best_to_file(args.outfile, metric, generated_samples_clean, "Sort by ratio of perplexity of Llama2-7B.Q4_K_M and Llama2-7B.Q3_K_S", "PPL-XL", scores["XL"], "PPL-SMALL", scores["SMALL"], lower_better=True) 
    print() 
    print() 
 
    # Sort by ratio of XL perplexity and ZLIB entropy 
    metric = np.log(scores["XL"]) / np.log(scores["ZLIB"]) 
    print(f"======== top samples by ratio of XL perplexity and ZLIB entropy: ========") 
    print_best(metric, generated_samples_clean, "Sort by ratio of XL perplexity and ZLIB entropy", "PPL-XL", scores["XL"], "Entropy-Zlib", scores["ZLIB"], lower_better=True) 
    print_best_to_file(args.outfile, metric, generated_samples_clean, "Sort by ratio of XL perplexity and ZLIB entropy", "PPL-XL", scores["XL"], "Entropy-Zlib", scores["ZLIB"], lower_better=True) 
    print() 
    print() 
 
    # Sort by ratio of perplexity of MODEL-XL on normal and lower-cased sample 
    metric = np.log(scores["XL"]) / np.log(scores["LOWER"]) 
    print(f"======== top samples by ratio of perplexity of GPT2-XL on normal and lower-cased sample: ========") 
    print_best(metric, generated_samples_clean, "Sort by ratio of perplexity of GPT2-XL on normal and lower-cased sample", "PPL-XL", scores["XL"], "PPL-XL-Lower", scores["LOWER"], lower_better=True) 
    print_best_to_file(args.outfile, metric, generated_samples_clean, "Sort by ratio of perplexity of GPT2-XL on normal and lower-cased sample", "PPL-XL", scores["XL"], "PPL-XL-Lower", scores["LOWER"], lower_better=True) 
    print() 
    print() 
 
 
if __name__ == "__main__": 
    parser = argparse.ArgumentParser() 
    parser.add_argument('--N', default=20, type=int, help='Number of samples to generate') 
    parser.add_argument('--batch_size', default=6, type=int, help='Batch size') 
    parser.add_argument('--outfile', default="./output/top_n_samples.txt", type=str, help='Output file to log top samples based on each metric') 
 
    args, _ = parser.parse_known_args() 
 
    main(args)

Loading models...


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models\llama-2-7b.Q3_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32            

Llama2-7B models loaded!


Generating samples:   0%|          | 0/20 [00:00<?, ?it/s]llama_perf_context_print:        load time =     639.98 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   255 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   22951.35 ms /   256 tokens
Llama.generate: 255 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     639.98 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   256 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   23136.61 ms /   257 tokens
llama_perf_context_print:        load time =    8865.20 ms
llama_perf_context_print: prompt eval tim

20
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
Num duplicates: 0
======== top samples by XL perplexity: ========
Metric Name: Sort by perplexity of Llama2-7B.Q4_K_M
1: PPL-XL=1.195, , score=0.178

('# Lukoil Arena\n'
 '\n'
 'Lukoil Arena (Russisch: ЛукОил Арена) is een multifunctioneel stadion in de '
 'Russische stad Nizjni Novgorod. Het is het stadion van de voetbalclub Volga '
 'Nizjni Novgorod. De naam is verwant aan de sponsor Lukoil.\n'
 '\n'
 'Het stadion werd geopend op 22 juli 2018. Het stadion kan 44.800 '
 'toeschouwers aan. Het stadion is ook in aanleg voor de handbaltoernooi op de '
 'Olympische Zomerspelen 2012. Het stadion staat in de omgeving van het '
 'Nizjnie-Novgorod Sports Complex. In 2018 is het stadion het thuisstadion van '
 'de Russische voetbalclub Volga Nizjni Novgorod. In 2017 werd de naam van het '
 'stadion al aangepast van Gorny stadion naar Lukoil Arena, dat gebeurde op 28 '
 

References:

https://arxiv.org/pdf/2012.07805

https://arxiv.org/pdf/2202.07646

https://github.com/shreyansh26/Extracting-Training-Data-from-Large-Langauge-Models

https://github.com/ftramer/LM_Memorization/blob/main/extraction.py

https://huggingface.co/TheBloke/Llama-2-7B-GGUF